In [1]:
# %pip install Pillow
# %pip install pydicom
# %pip install scipy

# %pip install shutil

# %pip install nibabel

# %pip install numpy

# %pip install imageio

# !chmod +x nii2png

# pip install nii2png as nii2

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

Getting No of Cases

In [2]:
df = os.listdir('/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train')
len(df)

585

Downloading the library for the conversion of dcm to niix

In [3]:
# !apt-get update
# !apt-get install -y dcm2niix
labels_df=pd.read_csv("/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")

print(labels_df.dtypes)  # Check data types
print(labels_df.head())  # Check sample data


BraTS21ID     int64
MGMT_value    int64
dtype: object
   BraTS21ID  MGMT_value
0          0           1
1          2           1
2          3           0
3          5           1
4          6           1


In [4]:
import os
import numpy as np
import pydicom
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd


# Split data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(images,label, test_size=0.2, random_state=42)

# Define image size and number of modalities
IMAGE_SIZE = (64, 64)
MODALITIES = ['T1w']  # List of modalities

# Load DICOM images from each modality
def load_dicom_images(data_dir,labels_df, output_path,modalities, target_size):
    images = []
    labels = []
    count=0
#     print(label_df)
        # Load from each modality
    for case in os.listdir(data_dir):
        case_id = str(case)  # Extract case ID from the folder name
        if len(case_id)<5:
            case_id=(5-len(case_id))*"0" + case_id
        print(case_id)
#         print(labels_df[labels_df['BraTS21ID'] == 00000]['MGMT_value'],labels_df[labels_df['BraTS21ID'] == "00000"])
        case_label = labels_df[labels_df['BraTS21ID'] == int(case_id)]['MGMT_value'].values
#         print(row)
#         filename = row['BraTS21ID']
#         label = row['MGMT_value']
#         print(index,label[0][0])
        # Construct the full path to the DICOM file
        file_path = os.path.join(data_dir,case)
        count+=1
        # Check if the file exists
        if os.path.exists(file_path):
            # Load the DICOM file
         
            for modalitie in modalities:
                new_file_path=os.path.join(file_path,modalitie)
                print(new_file_path)
#                 for file in os.listdir(new_file_path):
#                 print(new_file_path,file_path)
                for image in os.listdir(new_file_path):
                    dcm_path=os.path.join(new_file_path,image)
                    dicom_data = pydicom.dcmread(dcm_path)

                # Extract pixel data and convert to PIL Image
                    img = Image.fromarray(dicom_data.pixel_array)

                # Resize the image
                    img = img.resize(target_size)

                # Normalize the image (0-1 range)
                    img_array = np.array(img) / 255.0

                    images.append(img_array)
                    labels.append(case_label[0])
            else:
                print(f"File {dcm_path} not found. Skipping.")
                output=os.path.join(output_path,case)
                os.makedirs(output,exist_ok=True)
                !dcm2niix -z y -o "{output}" "{new_file_path}"

                       
        else:
            print(f"File {filename} not found.")
        print(data_dir,case,"done")

    return np.array(images), np.array(labels)
# Directory containing the DICOM data
data_dir = '/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/'
output_path='/kaggle/working/3D-mri/'
# Load the DICOM images from all modalities
images, label= load_dicom_images(data_dir,labels_df,output_path, MODALITIES, IMAGE_SIZE)

print(images.shape)


US1_J2KR.dcm:   0%|          | 38.0/154k [00:00<01:14, 2.05kB/s]
MR-SIEMENS-DICOM-WithOverlays.dcm:   0%|          | 125/511k [00:00<02:02, 4.16kB/s]
OBXXXX1A.dcm:   0%|          | 119/486k [00:00<01:53, 4.28kB/s]
US1_UNCR.dcm:   0%|          | 226/923k [00:00<02:36, 5.90kB/s]
color3d_jpeg_baseline.dcm:   0%|          | 1.50k/6.14M [00:00<05:34, 18.4kB/s]


00688
/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00688/T1w
File /kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00688/T1w/Image-168.dcm not found. Skipping.


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


/bin/bash: line 1: dcm2niix: command not found
/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/ 00688 done
00058
/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00058/T1w
File /kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00058/T1w/Image-17.dcm not found. Skipping.
/bin/bash: line 1: dcm2niix: command not found
/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/ 00058 done
00516
/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00516/T1w
File /kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00516/T1w/Image-31.dcm not found. Skipping.
/bin/bash: line 1: dcm2niix: command not found
/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/ 00516 done
00185
/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00185/T1w
File /kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00185/T1w/Image-168.dcm 

KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    images, label, test_size=0.2, random_state=42, stratify=label
)


In [6]:
import numpy as np

# # Save images and labels as .npy files
# np.save('X_train.npy', X_train)
# np.save('X_val.npy', X_val)
# np.save('y_train.npy', y_train)
# np.save('y_val.npy', y_val)

# print("Files saved successfully.")

X_train = np.load('/kaggle/working/X_train.npy')
X_val = np.load('/kaggle/working/X_val.npy')
y_train = np.load('/kaggle/working/y_train.npy')
y_val = np.load('/kaggle/working/y_val.npy')


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/X_train.npy'

Converting the niix to png image

In [ ]:
# def conver_niix_to_png(data_dire , output_path):
#     input_file=data_dire
#     main(sys.argv,input_file,output_path)
    


In [ ]:
# conver_niix_to_png("/kaggle/input/t1w-niix/3D-mri/00000/T1w_T1w_0_8.nii","/kaggle/working/")
# conver_niix_to_png("/kaggle/input/t1w-niix/3D-mri/00000/T1w_T1w_0_8.nii","/kaggle/working/")

Removing Pitch Black Images

In [ ]:
df=pd.read_csv('/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')

In [ ]:
grap=df['MGMT_value'].value_counts()
plt.figure(figsize=(8,5))
grap.plot(kind='bar')

In [ ]:
# Build a CNN model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 2nd Convolutional Layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 3rd Convolutional Layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten for fully connected layers
model.add(Flatten())

# Fully connected layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout to prevent overfitting

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(
    x=X_train,
    y=y_train,
    epochs=10,
    batch_size=126,
    validation_data=(X_val, y_val)
)

In [ ]:
# Plot training and validation accuracy/loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Save the model
model.save('multi_modality_tumor_detection_model.h5')